In [3]:
from os import listdir
from os.path import isfile, join

test_path = "../resources/testset"
dev_path = "../resources/devset"

test_files = [join(test_path,f) for f in listdir(test_path) if isfile(join(test_path, f))]
dev_files = [join(dev_path,f) for f in listdir(dev_path) if isfile(join(dev_path, f))]

In [21]:
class Document:
    def __init__(self, id, tokens, entities, sentences):
        self.id = id #book id
        self.tokens = tokens
        self.entities = entities
        self.sentences = sentences

class Entity:
    def __init__(self, id, ent_type, start_token, end_token):
        self.id = id #id in objects file
        self.ent_type = ent_type#location or locorg
        self.start_token = start_token
        self.end_token = end_token

class Sentence:
    def __init__(self, start_token, end_token):
        self.start_token = start_token
        self.end_token = end_token

In [33]:
import re

def generate_doc(doc_id, tokens_file, span_file, objects_file):
    tokens = []
    sentences = []
    entities = []
    spans = []
    
    current_start = 0
    current_id = 0
    for line in tokens_file.strip().split('\n'):
        if line == "":
            sentences.append(Sentence(current_start,current_id))
            current_start = current_id
            continue
        
        token = line.strip().split(' ')[-1]
        
        tokens.append(token)
        current_id += 1
    
    for line in span_file.strip().split('\n'):
        
        span_lst = line.strip().split(' ')
        entity_start = int(span_lst[4])
        entity_len = int(span_lst[5])
        entity_type = span_lst[1]
        span_id = int(span_lst[0])
        
        if entity_type != "loc_name" and entity_type != "loc_descr":
            continue
        
        spans.append((span_id, entity_start, entity_len))
    
    for line in objects_file.strip().split('\n'):
            object_lst = line.strip().split(' ')
            object_id = int(object_lst[0])
            object_type = object_lst[1]
            
            span_ids = []
            for i in range(2, len(object_lst)):
                sp_id = object_lst[i]
                if sp_id == '#':
                    break
                span_ids.append(int(sp_id))
                
            
            if object_type != "Location" and object_type != "LocOrg":
                continue
            
            entity_spans = []
            
            for span_id in span_ids:
                for span in spans:
                    if span[0] == span_id:
                        entity_spans.append(span)
                        break
            
            start = None
            max_len = 0
            
            for span_id, entity_start, entity_len in entity_spans:
                if entity_len > max_len:
                    max_len = entity_len
                    start = entity_start
            
            entities.append(Entity(object_id, object_type, start, start+max_len))
    return Document(doc_id, tokens, entities, sentences)   

def read_docs(files):
    re_pattern = "\d+"
    p = re.compile(re_pattern)
    
    files_with_id = dict()
    for f in files:
        file_id = int(p.search(f).group())
        lst = files_with_id.setdefault(file_id, list())
        lst.append(f)
    
    docs = []
    for id, lst in files_with_id.items():
        tokens = []
        entities = []
        
        span_file = None
        tokens_file = None
        objects_file = None
        
        for file in lst:
            with open(file, "r") as open_file:
                file_content = open_file.read()
                                      
            if file[-5:] == "spans":
                span_file = file_content
            elif file[-7:] == "objects":
                objects_file = file_content
            elif file[-6:] == "tokens":
                tokens_file = file_content
                
        docs.append(generate_doc(id, tokens_file, span_file, objects_file))
    
    return docs             
                     
  
docs = read_docs(dev_files)

TypeError: __init__() missing 1 required positional argument: 'end_token'